## Find each Franchise's rostered players

In [80]:
# import dependencies
import pandas as pd
from bs4 import BeautifulSoup
import requests
from datetime import date
import plotly.express as px
import plotly.graph_objects as go

# import config
from config import league_id, api_key_MFL

In [202]:
# Initialize Franchise data
franchise_df = pd.DataFrame()
franchise_df['Franchise'] = ['0001', '0002', '0003', '0004', '0005', '0006', '0007', '0008', '0009', '0010', '0011', '0012', 'FA']
franchise_df['FranchiseCode'] = ['PMS', 'WFF', 'VER', 'DWS', 'CRO', 'OHS', 'FNF', 'CAM', 'PBW', 'SAS', 'GUS', 'IDK', 'FA']
franchise_df['FranchiseName'] =['PMS', 
                                'WFF', 
                                'VER', 
                                'DWS', 
                                'CRO', 
                                'OHS', 
                                'FNF', 
                                'CAM', 
                                'PBW', 
                                'SAS', 
                                'GUS', 
                                'IDK', 
                                'FA'
                                ]


## Pull player data from MyFantasyLeague

In [203]:
# Get all players' name, team name, position
urlString = "https://api.myfantasyleague.com/2022/export?TYPE=players"
response = requests.get(urlString)
soup = BeautifulSoup(response.content,'xml')

data = []
players = soup.find_all('player')
for i in range(len(players)):
    rows = [players[i].get("id"), players[i].get("name"), players[i].get("position"), players[i].get("team")]
    data.append(rows)
player_df = pd.DataFrame(data)
player_df.columns=['PlayerID','Name', 'Position', 'Team']


In [204]:
# Get franchise rosters
urlString = f"https://www54.myfantasyleague.com/2022/export?TYPE=rosters&L={league_id}"
response = requests.get(urlString)
soup = BeautifulSoup(response.content,'xml')

data = []
franchises = soup.find_all('franchise')
for i in range(0,len(franchises)):
    current_franchise = franchises[i].find_all('player')
    for j in range(0,len(current_franchise)):
        rows = [franchises[i].get("id"), franchises[i].get("week"), current_franchise[j].get("id"), current_franchise[j].get("status")]
        data.append(rows)
rosters_df = pd.DataFrame(data)

# Get Free Agents
urlString = f"https://www54.myfantasyleague.com/2022/export?TYPE=freeAgents&L={league_id}"
response = requests.get(urlString)
soup = BeautifulSoup(response.content,'xml')

data = []
freeAgents = soup.find_all('player')
for i in range(len(freeAgents)):
    rows = ["FA", "", freeAgents[i].get("id"), "Free Agent"]
    data.append(rows)
fa_df = pd.DataFrame(data)
rosters_df = rosters_df.append(fa_df)
rosters_df.columns=['Franchise','Week','PlayerID','RosterStatus']



In [205]:
# Get Shark Ranks
urlString = "https://api.myfantasyleague.com/2022/export?TYPE=playerRanks"
response = requests.get(urlString)
soup = BeautifulSoup(response.content,'xml')

data = []
sharkRanks = soup.find_all('player')
for i in range(len(sharkRanks)):
    rows = [sharkRanks[i].get("id"), sharkRanks[i].get("rank")]
    data.append(rows)
shark_df = pd.DataFrame(data)
shark_df.columns=['PlayerID','SharkRank']
shark_df['SharkRank'] = shark_df['SharkRank'].astype('int32')


In [206]:
# Get adp
urlString = "https://api.myfantasyleague.com/2022/export?TYPE=adp"
response = requests.get(urlString)
soup = BeautifulSoup(response.content,'xml')

data = []
players = soup.find_all('player')
for i in range(len(players)):
    rows = [players[i].get("id"), players[i].get("averagePick")]
    data.append(rows)
adp_df = pd.DataFrame(data)
adp_df.columns=['PlayerID','ADP']
adp_df['ADP'] = adp_df['ADP'].astype('float32')



In [207]:
# Merge all dfs
complete = player_df.merge(rosters_df, on='PlayerID').merge(franchise_df[['Franchise', 'FranchiseCode']], on='Franchise').merge(shark_df, on='PlayerID').merge(adp_df, on='PlayerID')
complete = complete.sort_values(by=['SharkRank'])
complete.reset_index(inplace=True, drop=True)

In [208]:
qbs_2022 = complete[complete['Position'] == "QB"]
qbs_2022.reset_index(inplace=True, drop=True)
rbs_2022 = complete[complete['Position'] == "RB"]
rbs_2022.reset_index(inplace=True, drop=True)
wrs_2022 = complete[complete['Position'] == "WR"]
wrs_2022.reset_index(inplace=True, drop=True)
tes_2022 = complete[complete['Position'] == "TE"]
tes_2022.reset_index(inplace=True, drop=True)
pks_2022 = complete[complete['Position'] == "PK"]
pks_2022.reset_index(inplace=True, drop=True)
defs_2022 = complete[complete['Position'] == "Def"]
defs_2022.reset_index(inplace=True, drop=True)

## Import standard relative values

In [209]:
qbs = pd.read_excel("RelativeValues2021.xlsx", sheet_name="QB")
rbs = pd.read_excel("RelativeValues2021.xlsx", sheet_name="RB")
wrs = pd.read_excel("RelativeValues2021.xlsx", sheet_name="WR")
tes = pd.read_excel("RelativeValues2021.xlsx", sheet_name="TE")
pks = pd.read_excel("RelativeValues2021.xlsx", sheet_name="PK")
defs = pd.read_excel("RelativeValues2021.xlsx", sheet_name="DEF")

qbs = qbs['Regressed']
rbs = rbs['Regressed']
wrs = wrs['Regressed']
tes = tes['Regressed']
pks = pks['Regressed']
defs = defs['Regressed']


In [210]:
# Merge dfs
qbs_2022 = pd.merge(qbs_2022, qbs, how="left", left_index=True, right_index=True)
rbs_2022 = pd.merge(rbs_2022, rbs, how="left", left_index=True, right_index=True)
wrs_2022 = pd.merge(wrs_2022, wrs, how="left", left_index=True, right_index=True)
tes_2022 = pd.merge(tes_2022, tes, how="left", left_index=True, right_index=True)
pks_2022 = pd.merge(pks_2022, pks, how="left", left_index=True, right_index=True)
defs_2022 = pd.merge(defs_2022, defs, how="left", left_index=True, right_index=True)
analyzed = pd.concat([qbs_2022, rbs_2022, wrs_2022, tes_2022, pks_2022, defs_2022])


In [211]:
analyzed.groupby('FranchiseCode').sum().sort_values(by='Regressed', ascending=False)

,SharkRank,ADP,Regressed
FranchiseCode,,,
PBW,7823,2003.609985,1042.656665
CRO,11559,2825.510010,1005.029989
SAS,8458,2088.699951,955.816661
WFF,7763,1851.560059,954.353323
DWS,9318,2332.469971,927.006664
CAM,8482,2428.379883,739.696660
VER,8508,2122.070068,709.580004
IDK,9160,2268.780029,695.533324
FNF,10114,2528.070068,641.513330


In [212]:
analyzed['Date'] = date.today()
priorWeeks = pd.read_csv("2022Values.csv", index_col=0)
priorWeeks['Date'] = pd.to_datetime(priorWeeks['Date'])
result = pd.concat([priorWeeks, analyzed], axis=0, ignore_index=True)
result.to_csv("2022Values.csv")

## Roster-builder

In [214]:
today = result[result['Date'] == date.today()].sort_values(by='Regressed', ascending=False, ignore_index=True)

In [215]:
qbs_rostered = today[today['Position'] == "QB"]
qbs_rostered.reset_index(inplace=True, drop=True)
rbs_rostered = today[today['Position'] == "RB"]
rbs_rostered.reset_index(inplace=True, drop=True)
wrs_rostered = today[today['Position'] == "WR"]
wrs_rostered.reset_index(inplace=True, drop=True)
tes_rostered = today[today['Position'] == "TE"]
tes_rostered.reset_index(inplace=True, drop=True)

In [216]:
qbs_top = qbs_rostered.sort_values(by='Regressed', ascending=False, ignore_index=True).groupby('FranchiseCode').head(1)
rbs_top = rbs_rostered.sort_values(by='Regressed', ascending=False, ignore_index=True).groupby('FranchiseCode').head(2)
wrs_top = wrs_rostered.sort_values(by='Regressed', ascending=False, ignore_index=True).groupby('FranchiseCode').head(3)
tes_top = tes_rostered.sort_values(by='Regressed', ascending=False, ignore_index=True).groupby('FranchiseCode').head(2)

qbs_remainder = qbs_rostered[~qbs_rostered['PlayerID'].isin(qbs_top['PlayerID'])].groupby('FranchiseCode').head(1)
rbs_remainder = rbs_rostered[~rbs_rostered['PlayerID'].isin(rbs_top['PlayerID'])].groupby('FranchiseCode').head(3)
wrs_remainder = wrs_rostered[~wrs_rostered['PlayerID'].isin(wrs_top['PlayerID'])].groupby('FranchiseCode').head(3)
tes_remainder = tes_rostered[~tes_rostered['PlayerID'].isin(tes_top['PlayerID'])].groupby('FranchiseCode').head(3)
                                                         
remainder = pd.concat([qbs_remainder, rbs_remainder, wrs_remainder, tes_remainder])
                             
top_remainders = remainder.sort_values(by='Regressed', ascending=False, ignore_index=True).groupby('FranchiseCode').head(3)
                             
fran_rost = pd.concat([qbs_top, rbs_top, wrs_top, tes_top, top_remainders])
fran_rost = fran_rost.sort_values(by='Regressed', ascending=False, ignore_index=True)


## Examine Data

In [217]:
fran_rank = fran_rost.groupby('FranchiseCode').sum().sort_values(by='Regressed', ascending=False)
fran_rank

,SharkRank,ADP,Regressed
FranchiseCode,,,
WFF,1150,675.360008,958.359996
SAS,1150,537.530002,950.063327
PBW,1749,736.989993,889.993335
CRO,997,688.689999,860.493324
VER,1549,658.099993,819.746670
DWS,1806,955.020014,746.723332
CAM,1293,1075.429996,728.720000
IDK,3590,1062.530001,689.059993
GUS,2195,978.899994,687.633327


In [218]:
sorter = fran_rank.index

fran_rost.FranchiseCode = fran_rost.FranchiseCode.astype("category")
fran_rost.FranchiseCode.cat.set_categories(sorter, inplace=True)
fran_rost.sort_values(["FranchiseCode"], inplace=True)

fig = px.bar(fran_rost, 
             x="FranchiseCode", 
             y="Regressed", 
             color="Position", 
             text='Name', 
             color_discrete_map={
                "RB": "#1033a6", #blue #1033a6 #0c2987
                "TE": "#1295ad", #teal #02687b #038097
                "WR": "#643fc1", #purple #4f22bc #643fc1
                "QB": "#f5d000"}, #gold #f5d000
             category_orders={
                 "Position": ["RB", "QB", "WR", "TE"]}
            )
fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.show()

In [201]:
fran_rost[fran_rost['FranchiseCode']=='CRO']

,PlayerID,Name,Position,Team,Franchise,Week,RosterStatus,FranchiseCode,SharkRank,ADP,Regressed,Date
10,14056,"Murray, Kyler",QB,ARI,0005,1,ROSTER,CRO,82,19.510000,163.463332,2022-05-03
24,13131,"Mixon, Joe",RB,CIN,0005,1,ROSTER,CRO,12,28.559999,120.933331,2022-05-03
28,14071,"Montgomery, David",RB,CHI,0005,1,ROSTER,CRO,15,54.889999,112.946666,2022-05-03
49,14073,"Jacobs, Josh",RB,LVR,0005,1,ROSTER,CRO,26,56.650002,72.216666,2022-05-03
61,14974,"Mooney, Darnell",WR,CHI,0005,1,ROSTER,CRO,53,84.389999,62.583333,2022-05-03
64,14143,"Knox, Dawson",TE,BUF,0005,1,ROSTER,CRO,120,88.980003,59.833332,2022-05-03
66,13163,"Godwin, Chris",WR,TBB,0005,1,ROSTER,CRO,55,54.709999,57.953332,2022-05-03
68,13672,"Gesicki, Mike",TE,MIA,0005,1,ROSTER,CRO,121,89.940002,57.026666,2022-05-03
73,14783,"Hurts, Jalen",QB,PHI,0005,1,ROSTER,CRO,161,40.369999,53.446667,2022-05-03
77,13635,"Moore, D.J.",WR,CAR,0005,1,ROSTER,CRO,59,49.700001,49.133334,2022-05-03


In [110]:
today[today['FranchiseCode']=='WFF']

,PlayerID,Name,Position,Team,Franchise,Week,RosterStatus,FranchiseCode,SharkRank,ADP,Regressed,Date
1928,13164,"Kupp, Cooper",WR,LAR,0002,1,ROSTER,WFF,4,13.870000,185.579997,2022-05-02
1797,5848,"Brady, Tom",QB,TBB,0002,1,ROSTER,WFF,73,58.619999,171.646664,2022-05-02
1931,14136,"Samuel, Deebo",WR,SFO,0002,1,ROSTER,WFF,12,26.400000,136.586668,2022-05-02
1804,10700,"Cousins, Kirk",QB,MIN,0002,1,ROSTER,WFF,127,63.889999,118.080001,2022-05-02
1851,15254,"Harris, Najee",RB,PIT,0002,1,ROSTER,WFF,17,14.310000,93.793333,2022-05-02
2052,11247,"Ertz, Zach",TE,ARI,0002,1,ROSTER,WFF,118,101.769997,59.833332,2022-05-02
1950,14102,"Metcalf, DK",WR,SEA,0002,1,ROSTER,WFF,57,39.980000,51.483333,2022-05-02
1864,14075,"Harris, Damien",RB,NEP,0002,1,ROSTER,WFF,49,95.559998,48.003336,2022-05-02
2122,0520,"Commanders, Washington",Def,WAS,0002,1,ROSTER,WFF,763,184.380005,43.353330,2022-05-02
1961,14845,"Davis, Gabriel",WR,BUF,0002,1,ROSTER,WFF,71,111.070000,30.103333,2022-05-02
